# Parameters

In [1]:
model_file          = "../models/europe.json"
load_series_file    = "data/europe_load_series.csv"
nuclear_series_file = "data/europe_nuclear_2016.csv"
border_flows_file   = "data/europe_border_flows_2016.csv"
output_dir          = "data/europe_opf_test"

"data/europe_opf_test"

# Network

In [2]:
using TemperateOptimalPowerFlow
using CSV, DataFrames, DataDrop

Precompiling TemperateOptimalPowerFlow
  ✓ TemperateOptimalPowerFlow
  1 dependency successfully precompiled in 2 seconds. 51 already precompiled.


Import the model:

In [3]:
network = import_model(model_file)

Dict{String, OrderedCollections.OrderedDict{String, Any}} with 3 entries:
  "gen"    => OrderedDict("1"=>Dict{String, Any}("entsoe_names"=>Any[], "model"…
  "branch" => OrderedDict("1"=>Dict{String, Any}("br_r"=>0.00767519, "rate_a"=>…
  "bus"    => OrderedDict("100"=>Dict{String, Any}("coord"=>Any[24.0161, 38.436…

In [4]:
for gen ∈ values(network["gen"])
    gen["max_ramp_rate"] = gen["aggregated_type"] == "coal" ? 2.0 : 0.0
end

# Nuclear production

Import actual time series for nuclear generators:

In [5]:
nuc_series_df = CSV.read(nuclear_series_file, DataFrame)
nuc_series_df.id .= string.(nuc_series_df.id);

Separate nuclear generators as non-dispatchable:

In [6]:
split_nondispatchable!(network, nuc_series_df.id)

Export the time series:

In [7]:
filter!(row -> string(row.id) in keys(network["gen_nd"]), nuc_series_df)

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,79,1970.42,1970.52,1970.62,1970.62,1970.22,1968.94,1969.83,1969.93,1970.42,1969.63,1967.76,1965.79,1964.41,1963.04,1962.25,1962.45,1963.13,1960.67,1962.94,1963.04,1964.02,1964.51,1966.97,1966.97,1966.78,1964.41,1964.61,1964.71,1964.61,1963.43,1962.64,1962.94,1962.74,1963.04,1961.95,1960.58,1961.07,1961.07,1961.17,1961.17,1960.87,1961.36,1963.04,1965.5,1969.34,1969.83,1969.53,1970.42,1969.24,1967.76,1968.15,1967.47,1967.17,1967.07,1966.88,1967.07,1967.37,1967.17,1964.61,1961.95,1961.76,1959.79,1959.59,1958.41,1958.31,1959.79,1961.36,1961.56,1962.54,1963.13,1964.22,1960.08,1956.05,1957.23,1957.43,1956.44,1953.68,1953.29,1952.6,1956.15,1967.37,1968.25,1970.22,1964.02,1960.08,1958.61,1963.43,1956.05,1945.32,1943.35,1957.33,1956.15,1957.52,1957.92,1958.21,1956.44,1956.15,1954.08,1955.26,⋯
2,87,1062.4,1062.3,1062.9,1062.9,1063.6,1063.1,1063.1,1063.3,1062.9,1059.7,1061.1,1061.1,1061.1,1061.6,1062.1,1062.5,1062.6,1062.9,1063.3,1062.5,1062.8,1063.1,1062.8,1063.5,1064.1,1063.7,1063.8,1064.1,1065.0,1064.1,1064.8,1064.5,1065.1,1065.0,1064.9,1064.8,1065.1,1064.7,1065.4,1065.1,1064.8,1065.2,1064.9,1065.2,1064.8,1065.2,1065.2,1064.9,1064.6,1064.6,1065.5,1065.7,1065.9,1065.3,1065.2,1064.7,1065.6,1065.0,1065.6,1065.2,1065.2,1064.9,1065.1,1065.1,1064.6,1064.9,1064.5,1064.1,1063.6,1063.9,1063.9,1063.3,1064.5,1063.6,1063.6,1062.9,1064.1,1063.5,1063.4,1063.6,1063.9,1063.7,1063.4,1062.6,1063.6,1063.1,1063.7,1063.2,1063.0,1063.1,1063.1,1062.7,1062.8,1063.7,1062.9,1063.6,1062.9,1064.2,1064.6,⋯
3,160,975.114,975.404,975.791,975.888,975.694,975.501,975.888,975.211,975.501,975.791,973.759,975.501,976.081,976.275,975.694,976.468,977.339,976.855,977.146,975.888,975.694,975.694,975.501,976.081,975.888,976.178,975.985,976.178,975.888,975.985,975.791,975.404,975.211,977.146,978.21,975.404,975.114,975.501,975.694,975.694,975.888,975.888,975.985,976.372,975.985,976.178,976.662,976.275,975.888,976.662,977.049,976.275,976.081,976.855,976.468,976.662,976.275,976.275,976.565,976.952,977.339,976.081,975.888,975.985,976.081,975.791,976.372,975.404,976.081,976.275,976.275,976.372,976.081,977.049,976.468,976.468,977.533,976.468,976.952,978.113,978.79,978.307,978.5,978.5,978.887,979.371,978.5,978.984,978.79,978.887,978.79,978.693,978.79,978.5,978.597,978.693,978.5,978.887,978.887,⋯
4,176,929.4,927.9,929.1,929.3,928.8,929.5,930.1,929.9,929.4,929.6,929.1,929.5,930.3,930.4,930.3,930.8,930.2,930.1,930.9,930.7,930.3,930.5,930.2,930.1,929.7,929.8,928.9,928.8,929.2,927.4,927.5,927.8,926.5,927.6,927.8,927.0,926.9,926.7,926.9,926.7,927.2,927.2,926.9,927.3,926.0,926.3,926.3,925.7,925.2,925.1,924.9,924.0,923.8,923.6,922.4,924.5,925.1,924.3,925.1,925.3,924.7,924.1,924.2,923.8,922.9,923.3,923.1,922.1,923.4,924.1,923.6,922.6,921.7,920.0,919.9,920.3,919.3,919.3,919.3,918.6,919.2,919.4,919.4,918.2,918.8,919.2,918.6,919.2,920.2,919.6,919.9,920.2,919.6,921.9,922.3,920.5,921.0,920.8,918.9,⋯
5,179,1914.31,1914

In [8]:
nuc_ids = nuc_series_df.id

44-element Vector{String}:
 "79"
 "87"
 "160"
 "176"
 "179"
 "209"
 "210"
 "232"
 "249"
 "256"
 "257"
 "265"
 "318"
 ⋮
 "723"
 "732"
 "751"
 "757"
 "782"
 "846"
 "908"
 "917"
 "921"
 "943"
 "950"
 "982"

In [9]:
nuc_series = Matrix(nuc_series_df[:, 2:end]) / 100.

44×8736 Matrix{Float64}:
 19.7042   19.7052   19.7062   19.7062   …  19.7042   19.7052   19.7052
 10.624    10.623    10.629    10.629       10.633    10.63     10.624
  9.75114   9.75404   9.75791   9.75888     10.4758    9.80338   9.75114
  9.294     9.279     9.291     9.293        9.299     9.296     9.296
 19.1431   19.1431   19.1431   19.1431      19.1431   19.1431   19.1431
 12.7429   12.736    12.477    12.7685   …  12.7764   12.7321   12.7311
 12.0824   12.0824   12.0787   12.0787      12.1026   12.1024   12.0844
 13.64     13.6173   13.553    13.53        13.6965   13.6178   13.635
 12.7271   12.7467   12.7437   12.7301      12.9856   12.9398   12.9096
 12.0348   12.0338   12.0309   12.023       12.0309   12.0328   12.0348
 12.8447   12.8466   12.8437   12.8437   …  12.8234   12.8251   12.8417
 14.0004   13.997    13.8992   13.7054      14.0317   14.0294   14.0088
 17.21     17.21     17.21     17.21        17.26     17.26     17.21
  ⋮                                      ⋱ 

# Loads

List of all countries in the network:

In [10]:
countries = Set([bus["country"] for bus ∈ values(network["bus"])])

Set{String} with 26 elements:
  "SI"
  "RO"
  "IT"
  "NL"
  "FR"
  "DK"
  "SK"
  "RS"
  "BG"
  "AT"
  "CH"
  "XX"
  "CZ"
  "ME"
  "DE"
  "MK"
  "BA"
  "BE"
  "AL"
  "LU"
  "HR"
  "ES"
  "GR"
  "HU"
  "PL"
  "PT"

### Compute expected production by country

Determine the total expected production:

In [11]:
expected_prod = Dict(country => 0.0 for country ∈ countries)

for gen ∈ values(network["gen"])
    expected_prod[gen["country"]] += gen["pexp"]
end

sort(expected_prod, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 277.804
  "PL" => 135.462
  "FR" => 128.144
  "IT" => 126.413
  "ES" => 77.2725
  "NL" => 59.6867
  "AT" => 57.6037
  "RO" => 48.9335
  "GR" => 48.2076
  "PT" => 42.3998
  "CZ" => 35.3887
  "RS" => 29.6354
  "BG" => 19.8427
  "BE" => 17.3597
  "BA" => 13.4466
  "CH" => 11.6612
  "HU" => 10.6986
  "SI" => 10.1473
  "SK" => 9.12782
  "DK" => 7.71758
  "HR" => 6.54311
  "MK" => 5.00832
  "AL" => 4.43366
  "LU" => 3.56473
  "XX" => 2.6395
  "ME" => 2.33425

### Compute import/export balance by country

Export balance by country:

In [12]:
border_flows_df = CSV.read(border_flows_file, DataFrame)

Row,country,out_flow
,String3,Float64
1,DE,5663.42
2,SI,127.709
3,HR,-717.664
4,BE,-725.658
5,ES,-1454.95
6,FR,3016.91
7,RS,167.457
8,DK,-282.057
9,BA,425.84


In [13]:
border_flows = Dict{String, Float64}()
for (country, flow) in eachrow(border_flows_df)
    border_flows[country] = flow / 100.0
end
border_flows["XX"] = 0.0

sort(border_flows, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "DE" => 56.6342
  "FR" => 30.1691
  "CZ" => 12.5113
  "RO" => 6.64282
  "PT" => 5.73057
  "BA" => 4.2584
  "PL" => 2.34598
  "BG" => 1.89876
  "RS" => 1.67457
  "SI" => 1.27709
  "AL" => 0.0488879
  "LU" => 0.0
  "XX" => 0.0
  "ME" => -0.309394
  "MK" => -1.36804
  "DK" => -2.82057
  "CH" => -5.13167
  "SK" => -5.37451
  "HR" => -7.17664
  "BE" => -7.25658
  "GR" => -8.38198
  "HU" => -9.63781
  "NL" => -9.9127
  "AT" => -9.93982
  "ES" => -14.5495
  "IT" => -41.3325

Check: total out flow adds up to zero

In [14]:
sum(values(border_flows))

1.865174681370263e-14

### Compute nuclear production by country

Nuclear production by country:

In [15]:
avg_nuclear = sum(nuc_series, dims=2) / size(nuc_series, 2)

nuclear_prod = Dict(country => 0.0 for country ∈ countries)
for i = 1:length(nuc_ids)
    country = network["gen_nd"][nuc_ids[i]]["country"]
    nuclear_prod[country] += avg_nuclear[i]
end
nuclear_prod

Dict{String, Float64} with 26 entries:
  "SI" => 0.0
  "ES" => 45.3376
  "GR" => 0.0
  "LU" => 0.0
  "HU" => 16.8633
  "CH" => 22.9463
  "MK" => 0.0
  "RS" => 0.0
  "XX" => 0.0
  "DE" => 90.667
  "AL" => 0.0
  "FR" => 441.027
  "PL" => 0.0
  "IT" => 0.0
  "AT" => 0.0
  "HR" => 0.0
  "PT" => 0.0
  "ME" => 0.0
  "NL" => 4.22129
  "BE" => 46.9143
  "RO" => 11.829
  "BG" => 17.1781
  "CZ" => 24.8885
  "SK" => 16.7945
  "DK" => 0.0
  "BA" => 0.0

### Compute total load by country

Determine annual load by country:

In [16]:
total_load = Dict(country => prod + nuclear_prod[country] - border_flows[country]
    for (country, prod) ∈ expected_prod)

sort(total_load, byvalue=true, rev=true)

OrderedCollections.OrderedDict{String, Float64} with 26 entries:
  "FR" => 539.002
  "DE" => 311.837
  "IT" => 167.745
  "ES" => 137.16
  "PL" => 133.116
  "NL" => 73.8207
  "BE" => 71.5306
  "AT" => 67.5436
  "GR" => 56.5896
  "RO" => 54.1198
  "CZ" => 47.7659
  "CH" => 39.7391
  "HU" => 37.1997
  "PT" => 36.6692
  "BG" => 35.122
  "SK" => 31.2968
  "RS" => 27.9608
  "HR" => 13.7197
  "DK" => 10.5382
  "BA" => 9.18824
  "SI" => 8.87024
  "MK" => 6.37635
  "AL" => 4.38477
  "LU" => 3.56473
  "ME" => 2.64364
  "XX" => 2.6395

In [17]:
# total_expected_load = sum(values(expected_load_by_country))

### Import loads

In [18]:
normalized_loads_df = CSV.read(load_series_file, DataFrame)
normalized_loads_df.id .= string.(normalized_loads_df.id);
sort!(normalized_loads_df, :id)

Row,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,⋯
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,100,0.752768,0.717142,0.696277,0.702589,0.740294,0.804763,0.871773,0.957464,1.03828,1.09663,1.12729,1.12028,1.0549,1.05845,1.06742,1.1151,1.19346,1.20978,1.208,1.14849,1.05499,0.990076,0.919315,0.824256,0.799082,0.754126,0.736063,0.747911,0.803706,0.889586,0.981578,1.06507,1.11263,1.1595,1.20158,1.19006,1.11759,1.11425,1.12159,1.16984,1.26288,1.26085,1.25182,1.18341,1.08118,1.02273,0.915134,0.823659,0.809845,0.776498,0.754136,0.773753,0.869332,1.00476,1.10817,1.18628,1.19218,1.17897,1.16665,1.15526,1.1184,1.12345,1.13368,1.22008,1.36623,1.38881,1.37606,1.29643,1.18416,1.09944,1.01156,0.904112,0.891334,0.869298,0.856456,0.865506,0.933155,1.05196,1.12349,1.1924,1.22341,1.23117,1.23177,1.23479,1.16618,1.17498,1.20098,1.28389,1.40624,1.42699,1.41708,1.33966,1.22423,1.13783,1.02218,0.940043,0.929715,0.887716,0.871619,⋯
2,101,0.664876,0.647517,0.625364,0.633201,0.682936,0.750584,0.828085,0.91222,0.966996,0.985308,0.993957,0.99123,0.951054,0.935229,0.930786,0.963302,1.04814,1.06175,1.05317,1.01434,0.932692,0.883219,0.835676,0.751556,0.734347,0.712446,0.697266,0.709676,0.763499,0.817529,0.879333,0.936023,0.961053,0.980422,0.987862,0.979177,0.933788,0.93186,0.924046,0.994046,1.08807,1.10365,1.09088,1.06321,0.987635,0.938112,0.870963,0.780312,0.767694,0.742718,0.725597,0.739868,0.786713,0.856628,0.952193,1.05841,1.11375,1.13141,1.13469,1.10741,1.03308,1.01244,0.99597,1.06179,1.16939,1.17788,1.15363,1.10276,1.01972,0.946231,0.86736,0.781654,0.766982,0.743633,0.728852,0.743891,0.808951,0.899322,1.00129,1.07464,1.11941,1.13146,1.14301,1.14446,1.10242,1.10687,1.09759,1.12531,1.1968,1.19051,1.16206,1.09586,1.00569,0.939689,0.863981,0.786707,0.765093,0.73416,0.715275,⋯
3,1011,0.772377,0.745524,0.722364,0.713669,0.708768,0.719272,0.776597,0.851975,0.928459,0.972265,0.973946,1.00141,0.971396,0.945312,0.946959,0.935916,0.922838,0.968285,1.05184,1.0706,1.05746,1.0267,0.972504,0.907323,0.832176,0.790323,0.756005,0.7534,0.751071,0.757298,0.822027,0.924224,1.02658,1.08968,1.11427,1.13421,1.10215,1.07141,1.09161,1.07065,1.05396,1.04166,1.10597,1.127,1.11303,1.08673,1.05014,0.962736,0.886605,0.842429,0.812248,0.79369,0.784898,0.794737,0.846343,0.980066,1.08337,1.13027,1.10911,1.11307,1.04641,1.02182,1.0437,1.02964,1.004,1.00651,1.0681,1.08033,1.06542,1.02911,1.00823,0.9271,0.867385,0.815647,0.783863,0.772697,0.764865,0.780478,0.838239,0.955475,1.05862,1.09807,1.09683,1.13216,1.09097,1.06071,1.10557,1.08954,1.07083,1.03622,1.06253,1.07526,1.05353,1.01394,1.00611,0.923373,0.863746,0.823675,0.795709,⋯
4,103,0.857148,0.817441,0.784019,0.783994,0.814874,0.868933,0.937451,1.03269,1.10925,1.16234,1.1969,1.18086,1.1152,1.11625,1.12299,1.1634,1.26023,1.27124,1.26857,1.22051,1.14111,1.07294,1.00204,0.889407,0.875078,0.834408,0.807967,0.818881,0.873586,0.961753,1.06152,1.1594,1.21294,1.25815,1.28953,1.28019,1.21295,1.22155,1.22272,1.2668,1.3625,1.37082,1.35313,1.2951,1.20467,1.13388,

Check that the IDs match:

In [19]:
bus_ids = normalized_loads_df.id
bus_ids == collect(keys(network["bus"]))

true

In [20]:
normalized_loads = Matrix(normalized_loads_df[:, 2:end])

4097×8736 Matrix{Float64}:
 0.752768  0.717142  0.696277  0.702589  …  0.923474  0.841104  0.76039
 0.664876  0.647517  0.625364  0.633201     0.806985  0.74257   0.668211
 0.772377  0.745524  0.722364  0.713669     0.960242  0.91967   0.855263
 0.857148  0.817441  0.784019  0.783994     1.05532   0.964578  0.8684
 0.816078  0.785923  0.760716  0.76332      1.01055   0.901311  0.824303
 0.783083  0.759596  0.739883  0.753484  …  0.940014  0.882779  0.791839
 0.840348  0.808831  0.791357  0.809653     1.0189    0.95853   0.845591
 0.769512  0.741986  0.718114  0.722905     0.908815  0.8597    0.77639
 1.09074   0.992256  0.940819  0.92412      1.46904   1.38348   1.21887
 0.922043  0.890897  0.870482  0.887219     1.14026   1.04651   0.938114
 0.883047  0.844995  0.824661  0.838603  …  1.08491   0.992258  0.913734
 0.725686  0.698746  0.687611  0.692334     0.879622  0.777091  0.723823
 0.830083  0.788734  0.770518  0.764617     1.01902   0.915549  0.841104
 ⋮                           

Check normalization:

In [21]:
sum(normalized_loads) / length(normalized_loads)

1.0001398557038088

Normalization by country:

In [22]:
normalized_annual_loads = Dict(country => [] for country ∈ countries)
for (id, annual_load) ∈ Dict(bus_ids .=> sum(normalized_loads, dims=2) / size(normalized_loads, 2))
    push!(normalized_annual_loads[network["bus"][id]["country"]], annual_load)
end

normalization = Dict(country => sum(annual_loads) / length(annual_loads)
    for (country, annual_loads) ∈ normalized_annual_loads)

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.00292
  "HU" => 0.999803
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.00069
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.00175
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.00018
  "BE" => 1.00039
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

Check that the load assignement within each country is properly normalized:

In [23]:
load_prop_by_country = Dict(country => 0.0 for country in countries)
for bus ∈ values(network["bus"])
    load_prop_by_country[bus["country"]] += bus["load_prop"]
end

load_prop_by_country

Dict{String, Float64} with 26 entries:
  "SI" => 1.0
  "ES" => 1.0
  "GR" => 1.0
  "LU" => 1.0
  "HU" => 1.0
  "CH" => 1.0
  "MK" => 1.0
  "RS" => 1.0
  "XX" => 1.0
  "DE" => 1.0
  "AL" => 1.0
  "FR" => 1.0
  "PL" => 1.0
  "IT" => 1.0
  "AT" => 1.0
  "HR" => 1.0
  "PT" => 1.0
  "ME" => 1.0
  "NL" => 1.0
  "BE" => 1.0
  "RO" => 1.0
  "BG" => 1.0
  "CZ" => 1.0
  "SK" => 1.0
  "DK" => 1.0
  "BA" => 1.0

Compute a scaling value for each country:

In [24]:
load_scaling = [(
        network["bus"][id]["load_prop"]
        * normalization[network["bus"][id]["country"]]
        * total_load[network["bus"][id]["country"]]
        )
    for id ∈ bus_ids]

4097-element Vector{Float64}:
 0.8934481536600731
 1.5825390799958012
 0.8131218182721368
 1.4753994931380312
 2.2254772016934594
 0.43722182396868503
 0.8807907339942219
 0.9598877612797007
 0.07712152835090172
 1.8597526716908297
 3.4264550316060913
 3.470974293118438
 3.5007212076226675
 ⋮
 0.12830896653476662
 0.19865342615385437
 0.39730685217527295
 0.21543870091455344
 0.19136704331302068
 0.47195804130982344
 0.9439160823050085
 0.12205690440108899
 0.19904456890544608
 0.39808913767819576
 0.2071871502124192
 1.4718375317669001

In [25]:
loads = load_scaling .* normalized_loads

4097×8736 Matrix{Float64}:
 0.672559   0.640729   0.622087   0.627727   …  0.825076  0.751483  0.679369
 1.05219    1.02472    0.989662   1.00206       1.27708   1.17515   1.05747
 0.628037   0.606202   0.58737    0.5803        0.780794  0.747804  0.695433
 1.26464    1.20605    1.15674    1.1567        1.55702   1.42314   1.28124
 1.81616    1.74905    1.69296    1.69875       2.24895   2.00585   1.83447
 0.342381   0.332112   0.323493   0.32944    …  0.410995  0.38597   0.346209
 0.74017    0.712411   0.69702    0.713135      0.897438  0.844264  0.744789
 0.738645   0.712224   0.689309   0.693907      0.872361  0.825215  0.745247
 0.0841195  0.0765243  0.0725574  0.0712696     0.113294  0.106696  0.0940009
 1.71477    1.65685    1.61888    1.65001       2.1206    1.94626   1.74466
 3.02572    2.89534    2.82567    2.87344    …  3.71741   3.39993   3.13087
 2.51884    2.42533    2.38668    2.40307       3.05315   2.69726   2.51237
 2.90589    2.76114    2.69737    2.67671       3.5673

Check that the annual load matches the expectations:

In [26]:
sum(loads) / size(loads, 2) / sum(values(total_load))

1.0003905144711407

# Generation costs

In [27]:
# gen_costs_df = CSV.read("../doc/data/europe_noise_series.csv", DataFrame)

In [28]:
# gen_costs = Matrix(gen_costs_df[gen_ids, 2:end])

# Setup

In [29]:
setup(output_dir, network, loads, nuc_ids, nuc_series, overwrite = true)

[2024-06-26 14:19:09]  Overwriting directory 'data/europe_opf_test'
[2024-06-26 14:19:09]  Loading network
[2024-06-26 14:19:09]  Computing susceptance matrix
[2024-06-26 14:19:10]  Computing PTDF matrix (this may take some time)
[2024-06-26 14:20:31]  Distributing generator on buses
[2024-06-26 14:20:31]  Distributing non-dispatchable generators as loads
[2024-06-26 14:20:31]  Computing max capacity for each generator
[2024-06-26 14:20:31]  Computing expected production for each generator
[2024-06-26 14:20:31]  Computing thermal limits
[2024-06-26 14:20:31]  Computing quadratic line costs
[2024-06-26 14:20:32]  Computing linear line costs
[2024-06-26 14:20:45]  Computing total load constraints
[2024-06-26 14:20:45]  Computing ramp constraints


In [30]:
readdir(output_dir)

14-element Vector{String}:
 "A_gen.h5"
 "A_nondispatch.h5"
 "A_ramp.h5"
 "PTDF_matrix.h5"
 "P_exp.h5"
 "P_load.h5"
 "P_max.h5"
 "P_nondispatch.h5"
 "P_total.h5"
 "linear_line_cost.h5"
 "quadratic_cost.h5"
 "ramp_max.h5"
 "susceptance.h5"
 "thermal_limits.h5"